In [ ]:
#L ← Empty list that will contain the sorted elements
#S ← Set of all nodes with no incoming edge

#while S is not empty do
#    remove a node n from S
#    add n to L
#    for each node m with an edge e from n to m do
#        remove edge e from the graph
#        if m has no other incoming edges then
#            insert m into S

#if graph has edges then
#    return error   (graph has at least one cycle)
#else 
#    return L   (a topologically sorted order)

import numpy as np

seed = 123
np.random.seed(seed)

def topsort(G):
    L = []
    S = []
    edges = list(G.edges())
    print(edges)
    nodes = list(G.nodes())
    predecessors = dict([])
    successors = dict([])
    for n in nodes:
        predecessors[n] = list(G.predecessors(n))
        successors[n] = list(G.successors(n))
        if len(predecessors[n])==0:
            S.append(n)
    #print(S)        
    while len(S)>0:
        #print(S,L)
        n = S.pop(0)
        L.append(n)
        succ = [x for x in successors[n]]
        
        for m in successors[n]:
            pred = [x  for x in predecessors[m] if x!=n]
            edges.remove((n,m))
            #succ.remove(m)
            
            if pred==[]:
                #print(m)
                succ.remove(m)
                S.append(m)
            predecessors[m] = [x for x in pred]    
        #print(succ)
        successors[n] = [s for s in succ]
    if len(edges)>0:
        #print(edges)
        print(len(L),L)
        return None
    return L

def getRandomAcyclicPath(G,maxlen=100,seed=123):
    
    
    np.random.seed(seed)
    path = []
    v = np.random.choice(G.nodes())
    while not ((v in path) and len(path)<=maxlen):
        if v in path:
            break
        path.append(v)
        scc = list(G.successors(v))
    #print(v,">",G[v],",",path)
        if len(scc)>0:
            v = np.random.choice(scc)
    return path


def deming_regression(xx,yy):
    import numpy as np
    
    n = len(xx)
    xhat = 1.0/n * sum(xx)
    yhat = 1.0/n * sum(yy)
    s2x = 1.0/(n-1)*sum([(xx[i]-xhat)**2 for i in range(len(xx))])
    s2y = 1.0/(n-1)*sum([(yy[i]-yhat)**2 for i in range(len(yy))])
    sxy = 1.0/(n-1)*sum([(yy[i]-yhat)*(xx[i]-xhat) for i in range(len(yy))])
    
    if abs(sxy)<0.01:
        sxy = 0.01
    if s2y**2-2*s2y*s2x+s2x**2+4*1*sxy**2 <0.0:
        sr = 0.0
    else:
        sr = np.sqrt(s2y**2-2*s2y*s2x+s2x**2+4*1*sxy**2)    
    
    beta1 = (s2y-1*s2x+sr)/(2*sxy)
    
    beta0 = yhat-beta1*xhat
    
    xstar = [xx[i]+beta1/(beta1**2+1)*(yy[i]-beta0-beta1*xx[i]) for i in range(len(xx))]
    return (beta1,beta0,xstar)


def pathCost(path,cost,verbose=False):
    s = 0
    for i in range(len(path)-1):
        u,v = path[i],path[i+1]
        s+=cost[(u,v)]
        if verbose: print("pathcost, u,v,cost = ",u,v,cost[(u,v)])
    return s    


def is_prime(n):
    for k in range(2,1+int(np.floor(np.sqrt(n)))):
        if n%k ==0 and 1 < k < n:
            return False
    return n!=1


def primes(n):
    return [x for x in range(1,n+1) if is_prime(x)]

def prime_divisors(n):
    pd = []
    for k in range(1,1+int(np.floor(np.sqrt(n)))):
        if n%k==0:
            if is_prime(k):
                pd.append(k)
            if is_prime(n//k) and k!=n//k:
                pd.append(n//k)
    return pd

def gcd(a,b):
    return int(np.gcd(a,b))

def valuation(n,p):
    v = 0
    while n%p==0:
        v += 1
        n = n//p
    return v    


def kk(a,b):
    return sum([valuation(a,p)*valuation(b,p) for p in prime_divisors(gcd(a,b))])

def dist(a,b):
    return np.sqrt(kk(a,a)+kk(b,b)-2*kk(a,b))


print([prime_divisors(n) for n in range(1,11)])

def GG(n,f=1,rnd=True):
    import numpy as np
    G = nx.DiGraph()
    for a in range(1,n+1):
        G.add_node(a)
    for a in range(1,n+1):        
        for p in primes(n//a):
            b = a*p
            G.add_edge(a,b,weight=np.round(dist(a,b),3))
            #if a%b==0 and is_prime(a/b):
            #    G.add_edge(a,b,abs(a-b))
    return G

def projectGraph(G,S,paths,verbose=False):
    #xx = [pos[a][0] for a in G.nodes()]
    #yy = [pos[a][1] for a in G.nodes()]
    
    rvec = 2**20 # Richtungsvektor der Geraden durch den Ursprung
    
    #x0,y0 = np.array([x0,y0])-img.shape[0]*rvec
    #x1,y1 = np.array([x1,y1])+img.shape[1]*rvec
    #rvec = np.array([x1-x0,y1-y0])
    newpos = dict([])
    gN = list(G.nodes())
    mapG = dict(zip(range(len(gN)), gN))
    for i in range(len(G.nodes())):
        # projektion auf Gerade durch Ursprung mit Richtungsvektor rvec
        newpos[mapG[i]] = kk(rvec,mapG[i])/kk(rvec,rvec) 
        

    cost = dict([])    
    tops = (list(nx.topological_sort(G)))
    
    xsorted = sorted([(kk(rvec,mapG[i]),mapG[i]) for i in range(len(G.nodes()))])
    isorted = [x[1] for x in xsorted]
    if verbose: print("isorted =", isorted)

    zipped = dict(list(zip(tops,isorted)))

    if verbose:
        print("topsort = ",tops)
        print("xsorted = ",xsorted)
        print("zipped = ", zipped)

            
        
    newcost = dict([])
    #cost = dict([])

    myG = nx.DiGraph()

    #dm = nx.diameter(G)
    
    minCost = np.inf
    
    for e in G.edges():
        u,v = e
        myG.add_node(u)
        myG.add_node(v)
        if not (u,v) in cost.keys():
            cost[(u,v)] = dist(u,v)
        if minCost>cost[(u,v)]:
            minCost = cost[(u,v)]
        if not (u in S.nodes() and v in S.nodes()): #if not (u,v) in S.edges():
            if verbose: print(" not in S (u,v) = ",u,v)
            newcost[(u,v)] = cost[(u,v)]
            myG.add_edge(u,v,weight=newcost[(u,v)])
        if u in S.nodes() and v in S.nodes():#if (u,v) in S.edges():
            if verbose: print("u,v = ",u,v)
            U,V = zipped[u],zipped[v]
            newcost[(u,v)] = abs(newpos[U]-newpos[V]) 
            myG.add_edge(u,v,weight=newcost[(u,v)])
            
    #return myG,newcost,newpos        

    # determine constant for paths    
    minPathDist = np.inf
    maxPathDist = -np.inf
    reweightFactor = np.inf
    eps = 0.001
    #for path in paths:
    #    u,v= path[0],path[-1]
    #    dist = nx.shortest_path_length(myG,u,v,weight="weight")
    #    sdist = pathCost(path,newcost)
    #    if abs(sdist-dist)>eps:
    #        if reweightFactor > dist/sdist:
    #            reweightFactor = dist/sdist
    inds = [v for v in S.nodes()]
    radius = abs(newpos[xsorted[0][1]]-newpos[xsorted[-1][1]])
    reweightFactor = minCost / radius
    
    if verbose:
        print("reweightFactor = ",reweightFactor)
        print("radius = ",radius)
        print("minCost = ",minCost)
            
    if True:
        for e in myG.edges():        
            u,v = e
            if u in S.nodes() and v in S.nodes():#
                if verbose :
                    print("reweighting: (u,v) = ",(u,v),newcost[(u,v)],newcost[(u,v)]*reweightFactor)
                nx.set_edge_attributes(myG,{(u,v):{"weight":newcost[(u,v)]*reweightFactor}})
                newcost[(u,v)]= newcost[(u,v)]*reweightFactor
        
    return myG,newcost,newpos

def smallGraph():
    G = nx.DiGraph()
    
    G.add_nodes_from(list(range(9+1)))
    
    G.add_edges_from(
        [ (0,1),
          (1,2),
          (2,3),
          (3,4),(4,5),
          (0,6),(6,5),
          (0,7),(7,4),
          (0,8),(8,3),
          (0,9),(9,2)
        ]
    )
    pos = {
        0: (0,0), 1 : (1,0),2:(1,-1),3:(-1,-1),4:(-1,1),5:(1,1),
        6:(1/2,1/2),7:(1/2,-1/2),8:(-1/2,-1/2),9:(-1/2,1/2)
    }
    pos = dict([(k,np.array(pos[k])) for k in pos.keys()])
    return G, pos

def checkShortestPathProperty(myG,S,cost,randomShortestPaths,paths,verbose=False):
    eps = 1e-4
    nrErrors = 0
    if verbose: 
        print("paths = ", paths)
    for path in randomShortestPaths:
        u,v= path[0],path[-1]
        dist = nx.shortest_path_length(myG,u,v,weight="weight")
        sdist = pathCost(path,cost)
        if abs(sdist-dist)>eps and path in paths:#and all([path[i] in S.vertices() for i in range(len(path))]):
            print("Error:",path, sdist,dist)
            nrErrors += 1
        elif abs(sdist-dist)>eps and not path in paths: #all([path[i] in S.vertices() for i in range(len(path))]):
            if verbose: print("not a shortest path as required:",path,sdist,dist)
        elif abs(sdist-dist)<=eps  and path in paths: #all([path[i] in S.vertices() for i in range(len(path))]):
            if verbose: print("shortest path as required:",path,sdist,dist)
        elif abs(sdist-dist)<=eps  and not path in paths: #all([path[i] in S.vertices() for i in range(len(path))]):
            if verbose: print("shortest path but not required:",path,sdist,dist)
    print("nrErrors = ",nrErrors)        
    return nrErrors
    

In [ ]:
# topologische Sortierung
import time
import networkx as nx
from copy import deepcopy 

datapoints = []
number_of_nodes  = []
graph_creation_times = []
graph_projection_times = []
random_path_times = []

nrnodes = [100,500,1000,5000,10**4,10**5,10**6]
nrnodes = [1000,2000,10**4]
verbose=True
printIt = True
for nrn in nrnodes:
    print("number of nodes = ",nrn)
    start = time.time()
    G = GG(nrn,f=10) #smallGraph() #GG(nrn)
    end = time.time()
    print("Graph created in time = ",end-start)
    Gs = deepcopy(G)
    #ox.plot_graph(G)
    #paths = [ getRandomAcyclicPath(G) for k in range(140)]
    H = Gs
    number_of_nodes.append(nrn)
    graph_creation_times.append(end-start)


    nrrndpaths = [0.2]
    for nrrnd in nrrndpaths:
        start = time.time()
        randomShortestPaths = [getRandomAcyclicPath(H,maxlen=100,seed=k+1) for k in range(int(np.floor(nrrnd*nrn)))]
        paths = randomShortestPaths[0:len(randomShortestPaths)//2]
        end = time.time()
        print("random paths chosen in time = ",end-start)
        random_path_times.append(end-start)
        datapoints.append((nrn,len(randomShortestPaths),len(paths)))
        edges = []
        verts = []
        for p in paths:
            if len(p)==1:
                continue
            for i in range(len(p)-1):
                verts.append(p[i])
                edges.append((p[i],p[i+1]))
            if len(p)>1: 
                verts.append(p[i+1])    
        S = nx.DiGraph()
        S.add_nodes_from(verts)
        S.add_edges_from(edges)

        #wcc = list(nx.weakly_connected_components(T))
        #print(wcc)
        #if len(wcc)==0:
        #    S = DiGraph(T)
        #else:    
        #    comp = wcc[0]
        #    S= DiGraph(nx.subgraph(T,comp))
        
  

        print("projecting graph")
        start = time.time()
        myG,newcost,newpos = projectGraph(G,S,paths,verbose=verbose)
        end = time.time()
        graph_projection_times.append(end-start)

        print("graph projected in time = ",end-start)

        checkShortestPath = printIt
        if checkShortestPath:
            checkShortestPathProperty(myG,S,newcost,randomShortestPaths,paths,verbose=verbose)

        
print(datapoints)            
print(graph_creation_times)
print(random_path_times)
print(graph_projection_times)

In [ ]:
from copy import deepcopy



# hier werden die isotonische Kosten gespeichert
isoG = deepcopy(myG)

np.random.seed(123)


s = -1 # neuer Knoten der eingefuegt wird
nn = list(myH.nodes())

# hier werden die momentan verwendeteten Kosten gespeichert.
# Annahme: C(u,v) = randuniform(0.01,10)*chat(u,v)
myH = deepcopy(myG) 

for e in myH.edges():
    u,v = e
    nx.set_edge_attributes(myH,{(u,v):{"weight":np.random.uniform(0.001,100)*myG.get_edge_data(u,v)["weight"]}})


tops = list(nx.topological_sort(myH))

for v in nn:
    if myH.in_degree(v)==0:
        print(v)
        myH.add_node(s)
        myH.add_edge(s,v,weight=0)
        print(myH.get_edge_data(s,v))

       
        
hh = nx.single_source_bellman_ford_path_length(myH,s,weight="weight")

myH.remove_node(s)

hl = []
for v in tops:
    hl.append(hh[v])
print(hl)        

In [ ]:
from sklearn.datasets import make_regression
from sklearn.isotonic import IsotonicRegression



X = np.array([[i] for i in range(1,len(hl)+1)])
y = np.array(hl)
#X, y = make_regression(n_samples=10, n_features=1, random_state=41)
print(X)
print(y)
iso_reg = IsotonicRegression(increasing=True).fit(X, y)
hisotonic = [h for h in iso_reg.predict(X)]

print(hisotonic)

hiso = dict(zip(tops,hisotonic))

isotoniccost = {}
for e in myH.edges():
    u,v = e
    isotoniccost[(u,v)] = newcost[(u,v)]+hiso[v]-hiso[u]
    nx.set_edge_attributes(isoG,{(u,v):{"weight":isotoniccost[(u,v)]}})
    print("(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u = ",(u,v),newcost[(u,v)],isotoniccost[(u,v)],hiso[v]-hiso[u])

In [ ]:
checkShortestPathProperty(isoG,S,isotoniccost,randomShortestPaths,paths,verbose=verbose)

In [ ]:
lossIso = 0
lossPrevious = 0

for e in myH.edges():
    u,v = e
    costH = myH.get_edge_data(u,v)["weight"]
    isoCost = isoG.get_edge_data(u,v)["weight"]
    previousCost = myG.get_edge_data(u,v)["weight"]
    lossIso += (costH-isoCost)**2
    lossPrevious += (costH-previousCost)**2

sqrtLossIso = np.sqrt(lossIso)
sqrtLossPrevious = np.sqrt(lossPrevious)

print("sqrtLossIso = ",sqrtLossIso)
print("sqrtLossPrevious = ",sqrtLossPrevious)
    

In [ ]:
import pandas as pd

def omega(n):
    return len(prime_divisors(n))


rows = []
for k in range(len(datapoints)):
    row = []
    row.append(datapoints[k][0])
    nrNodes = int(datapoints[k][0])
    nrEdges = sum(omega(k) for k in range(1,nrNodes+1))
    row.append(nrEdges)
    row.append(datapoints[k][2])
    row.append(graph_creation_times[k//3])
    row.append(random_path_times[k])
    row.append(graph_projection_times[k])
    print(row)
    rows.append(row)
#print(rows)    
result = pd.DataFrame(columns=["nrNodesInGraph","nrEdgesInGraph","nrRndPaths","graphCreationTimeInSec","rndPathCreationTimesInSec","graphProjectionTimesInSec"],data=rows)    

print(result)

result.to_csv("results-topological-sorting-kernel.csv")
